# Washington Times News Scraper
### You need Google Chrome for running this notebook

#### When you already have Chrome installed:
1. Go to the website: https://sites.google.com/chromium.org/driver/downloads?authuser=0
2. Download a chrome driver that is the same version as your chrome
3. Double click the driver to open it
4. Come back here, start running the notebook

In [19]:
url = "https://www.theamericanconservative.com/web-categories/politics/"
# the dictionary for storing links of news
# dictionary can prevent duplicates
article_links = {}

In [60]:
# Necessary libraries
from bs4 import BeautifulSoup
from selenium import webdriver   
from requests import get
import time
import re
import csv
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
from selenium.webdriver.common.keys import Keys
import random
from selenium.common.exceptions import NoSuchElementException

In [30]:
# This snippet will open a Washington Times page on Chrome
# DON'T CLOSE THE WINDOW!
driver = webdriver.Chrome('/Users/wufangzheng/Downloads/chromedriver') #set the webdriver to Chrome driver
driver.get(url)  

## Web-scrape links of news

In [31]:
# The page can be infinitely scrolled down.
# Keep scrolling the page and store LINKS of the politics news in article_links

# This cell can fail every once in a while. Just re-run the cell when it fails.
listing = driver.find_element_by_class_name("alm-listing")
last_length = 0
for i in range(10000):
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.END)
    
    time.sleep(2)
    if i%20==0:
        links = listing.find_elements_by_class_name("alm-item")
        if len(links)== last_length:
            html.send_keys(Keys.PAGE_UP)
            html.send_keys(Keys.PAGE_DOWN)
            
            print("Not auto loading")
            continue
        else:
            last_length = len(links)
        count = 0
        for item in links:
            link = item.find_element_by_tag_name("a").get_attribute("href")
            if link not in article_links:
                article_links[link] = 0
                count+=1
        print(str(i) + " " + str(count))

0 0
20 0
40 0
60 0
80 0
100 0
120 0
140 0
160 0
180 0
200 0
220 0
240 0
260 0
280 0
300 0
320 0
340 0
360 0
380 0
400 0
420 0
440 0
460 0
480 0
500 0
520 25
540 100
560 100
580 100
600 100
620 100
640 100
660 84
Not auto loading


KeyboardInterrupt: 

## Web-scrape article text from those links

In [35]:
links = list(article_links.keys())

In [62]:
# Open every link from the links, and web-scrape text from it, and store the data to FOX_articles.csv

# The data we scrape: title, text (article body), date
# The output data will have the format of: index, date, title, text, label(left/lean left/neutral/lean right/right), link

# Note that I have to use selenium to scrape content as this website prevents from Beautiful Soup scraping.

# This cell may also fail
# Every time it fails, just re-run this cell. It will automatically continue with the remaining links

driver = webdriver.Chrome('/Users/wufangzheng/Downloads/chromedriver') #set the webdriver to Chrome driver
current_time = time.time()
difference = real_count - count
start_point = real_count
count = start_point-difference - 1
for link in links[start_point:]:
    real_count +=1
    count+=1
#     if count>3:
#         break

    # Open the link in a Chrome window
    driver.get(link)
    time.sleep(1)
    
    # Find content of the article
    try:
        article = driver.find_element_by_class_name("c-single-blog__content")
    except NoSuchElementException:
        print(link)
        print("This article has no content")
        count-=1
        continue

    if not article:
        count-=1
        print("This link does not contain article:")
        print(link)
        continue
        
    # Find paragraphes
    paras = article.find_elements_by_tag_name("p")
    text = ""
    for item in paras:
        text+=item.text+"\n"
#     print(text)

    # Find title
    title = driver.find_element_by_class_name("c-single-blog__title")
    
    title = title.text
        
    # Find date
    date = driver.find_element_by_class_name("c-post-meta__date").text
    date = convertdate(date)
    if date=="error":
        count-=1
        print("error date!")
        print(link)
        continue
#     date = soup.find('div', class_ = "article-body__date-source")
#     if not date:
#         date = soup.find('div', class_="article-hero-headline__timestamp")
#     date = date.find('time')['datetime']
#     date = convertdate(date)
#     if date == "error":
#         print("date is not scraped")
#         print(link)
#         count-=1
#         continue

    # Store data
    with open("Washington_times_articles.csv", "a") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([str(count),
                         date,
                         title,
                         text,
                         "lean right",
                         link])
    if count%200 == 0:
        print(count)
        print("Time Spent: " + str(time.time()-current_time))
        current_time = time.time()
driver.close()

https://www.theamericanconservative.com/articles/lessons-from-ukraine/
This article has no content
https://www.theamericanconservative.com/articles/local-and-global/
This article has no content
https://www.theamericanconservative.com/articles/the-peace-coalition/
This article has no content
https://www.theamericanconservative.com/articles/capitalisms-balance/
This article has no content
https://www.theamericanconservative.com/articles/from-pushkin-to-putin/
This article has no content
https://www.theamericanconservative.com/articles/big-government-little-tea-party/
This article has no content
https://www.theamericanconservative.com/articles/libertarianisms-limits/
This article has no content
https://www.theamericanconservative.com/articles/iran-is-ready-for-a-deal/
This article has no content
3000
Time Spent: 345.0810749530792
3200
Time Spent: 482.9792912006378


In [63]:
print(count)
print(real_count)
print(links[real_count-1])

3288
3299
https://www.theamericanconservative.com/articles/whose-war/


In [49]:
# The function used when scraping date from websites
def convertdate(datestr):
    datestr = datestr.split(", ")
    year = datestr[1]
    month = datestr[0].split(" ")[0]
    day = datestr[0].split(" ")[1]
    if month == "January".upper():
        month = "01"
    elif month == "February".upper():
        month = "02"
    elif month == "March".upper():
        month = "03"
    elif month == "April".upper():
        month = "04"
    elif month == "May".upper():
        month = "05"
    elif month == "June".upper():
        month = "06"
    elif month == "July".upper():
        month = "07"
    elif month == "August".upper():
        month = "08"
    elif month == "September".upper():
        month = "09"
    elif month == "October".upper():
        month = "10"
    elif month == "November".upper():
        month = "11"
    elif month == "December".upper():
        month = "12"
    else:
        print("Error!")
        return "error"
    return year+"/"+month+"/"+day

In [ ]:
count = 0
real_count = 0
for link in article